In [1]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import sys, os, argparse

In [2]:
server_cpu = pd.read_csv("dataset/Intel-Datacenter-CPU.csv")
server_gpu = pd.read_csv("dataset/NVIDIA-Datacenter-GPU.csv")
desktop_cpu = pd.read_csv("dataset/Intel-Desktop-CPU.csv")
desktop_gpu = pd.read_csv("dataset/NVIDIA-Desktop-GPU.csv")

desktop_cpu

,Product,Release Date,Process Size (nm),TDP (W),DieSizeValue,Emb CFP,Ope CFP,Tot CFP,ECFPA,OCFPA,Single Core Score,Multi-Core Score
0,Core i7-3940XM,2012,22,55,160,1.835771,29.854936,31.690707,14.208270,22.867,645.0,2153.0
1,Core i7-4930K,2013,22,130,257,3.386104,70.566211,73.952316,15.300511,33.649,655.0,3305.0
2,Core i7-5960X,2014,22,140,356,5.407292,75.994382,81.401673,16.441889,26.160,1323.0,2508.0
3,Core i7-6950X,2016,14,140,246,3.647846,91.806630,95.454476,17.095478,30.537,1318.0,8603.0
4,Core i7-8700K,2017,14,95,154,2.019337,62.297356,64.316693,15.918552,33.100,1610.0,6511.0
5,Core i9-9900KS,2019,14,127,180,2.443075,83.281729,85.724803,16.245756,37.858,1813.0,8639.0
6,Core i9-10900K,2020,14,125,206,2.895031,81.970205,84.865237,16.579640,32.559,1760.0,9063.0
7,Core i9-11900KF,2021,14,125,276,4.257984,81.970205,86.228189,17.466505,24.301,2329.0,10058.0
8,Core i9-12900KF,2021,10,125,215,3.904141,90.653077,94.557219,22.665533,28.009,2589.0,15062.0
9,Core i9-13900KF,2022,10,125,257,4.912426,90.653077,95.565503,23.420813,23.433,2918.0,19659.0


In [3]:
# Generate the probabilistic modeled parameters for Carbon Footprint calculation
# Assume 10k sample size for each parameter
sample_size = 10000
carbon_intensity_distribution = [0.03125,0.03125,0.03125,0.03125,0.01041667,0.01041667,0.01041667,0.01041663,0.01041667,
                    0.01041667,0.01041667,0.01041667,0.01041667,0.01041667,0.01041667,
                    0.01041667,0.04166667,0.04166667,0.04166667,0.04166667,
                    0.02083333,0.02083333,0.02083333,0.02083333,0.0625,0.0625,
                    0.0625,0.0625,0.0625,0.0625,0.0625,0.0625]
carbon_intensity_sequence = np.random.choice(np.arange(480.8486,546.7014,2.057901), size=sample_size, replace=True, p = carbon_intensity_distribution)

gpa_mean=150
gpa_std=30

gpa_sequence = np.random.normal(gpa_mean, gpa_std, sample_size)
gpa_sequence = np.clip(gpa_sequence,50,300)


# We use the estimated reference values of defective density and EPA in very process node to model the complete sequence 
defective_density_reference =  {"7": 0.2, "10": 0.11, "14":0.09, "22": 0.08, "28":0.07, "65":0.05}
epa_reference =  {"7": 2.15, "10": 1.475, "14":1.2, "22": 1.2, "28":0.9}

epa_distribution = [0.092478422,0.09864365,
                    0.101726264,0.103575832,0.101726264,0.097410604,
                    0.091245376,0.08323058,0.073982737,0.061652281,
                    0.048088779,0.033908755,0.012330456,]

defective_density_distribution=[0.1125,0.1125,0.1125,0.1125,0.1125,0.05,0.05,0.05,0.05,0.05,
    0.0125,0.0125,0.0125,0.0125,0.0125,0.0125,0.0125,0.0125,
    0.0125,0.0125,0.0125,0.0125,0.0125,0.0125,0.0125,]

def probabilistic_model(reference, p, stride = 25, d1 = None, d2 = None):
    d1 = 0.11 - 0.095
    d2 = 0.42 - 0.11
    defect_distribution = dict()
    for node in reference:
        defect_bench = defective_density_reference[node]
        start = defect_bench - d1
        end = defect_bench + d2
        step = (end - start) / stride
        defect_distribution[node] = np.random.choice(np.arange(start, end, step), size=sample_size, replace=True, p=p)
    return defect_distribution


def highest_probability(result):
    unique_values, counts = np.unique(result, return_counts=True)
    probabilities = counts / len(result)  # Convert frequency models to probabilities
    # Find the maximum probability and its corresponding value
    max_prob_index = np.argmax(probabilities)
    max_prob_value = unique_values[max_prob_index]
    max_prob = probabilities[max_prob_index]

    return max_prob_value, max_prob


In [4]:
# Add the eco_chip_enhanced folder to the system path
sys.path.append(os.path.join(os.getcwd(), "../eco_chip_enhanced/"))

# Import the eco_chip function from the eco_chip_func module
from eco_chip_func import eco_chip

def get_cfp(process_node, power, area, defective_density, gpa, carbon_intensity, epa):
    # Calculate the Carbon Footprint (CFP) for the given processor
    args = argparse.Namespace(
        design_dir = 'testcases/CFP_survey/',
        chip_area = area,
        chip_power = power,
        node = process_node,
        defect_density = defective_density,
        gpa = gpa, 
        epa = epa,
        num_lifetime=None,
        tech_scaling_path='../eco_chip_enhanced/',
        carbon_intensity = carbon_intensity
    )

    # we have design carbon, manufacturing carbon, operational carbon and total carbon
    design_carbon, mfg_carbon, ope_carbon, total_carbon = eco_chip(args)
    embodied_carbon = float(design_carbon) + float(mfg_carbon)
    operational_carbon = float(ope_carbon)
    total_carbon = float(total_carbon)
    
    return embodied_carbon, operational_carbon, total_carbon


In [5]:
# lets plot the actual modeled distribution of all the parameters mentioned above with sample process node = 7nm
defective_density_sequence = probabilistic_model(defective_density_reference, defective_density_distribution)
epa_sequence = probabilistic_model(epa_reference, epa_distribution, stride = 13, d1 = 0.575, d2 = 0.025)

def calculate_mean_cfp(idx, proc):
    node = int(proc['Process Size (nm)'])
    power = int(proc['TDP (W)'])
    area = int(proc['DieSizeValue'])
    
    embodied_carbon_list = np.zeros(sample_size)
    operational_carbon_list = np.zeros(sample_size)
    
    for i in range(sample_size):
        emb_c, ope_c, tot_c = get_cfp(node, power, area, defective_density_sequence[str(node)][i],
                                        gpa_sequence[i], carbon_intensity_sequence[i], epa_sequence[str(node)][i])
        embodied_carbon_list[i] = emb_c
        operational_carbon_list[i] = ope_c
        
        
    embodied_carbon_mean = embodied_carbon_list.mean()
    operational_carbon_mean = operational_carbon_list.mean()
    total_carbon_mean = embodied_carbon_mean + operational_carbon_mean
    
    
    return idx, embodied_carbon_mean, operational_carbon_mean, total_carbon_mean   

In [7]:
desktop_gpu

,Unnamed: 0,Product,Release Date,Process Size (nm),TDP (W),DieSizeValue,Die Size (mm^2),Transistors (million),Emb CFP,Ope CFP,Tot CFP,ECFPA,OCFPA,OpenCL Score,3D Mark Score,Price
0,0,TITAN BLACK,2014,28,230,561,561,7080,7.701342,104.296901,112.367484,8.359525,185.91248,30034,4179,999
1,1,TITAN X,2015,28,250,601,601,8000,8.474066,113.366197,122.235830,9.132249,188.62929,41486,5305,999
2,2,TITAN Pascal,2016,14,250,471,471,12000,8.041427,169.710334,179.171502,11.055738,360.31915,68312,9548,1199
3,3,TITAN Xp,2017,14,250,471,471,12000,8.041427,169.710334,179.171502,11.055738,360.31915,72482,10136,1199
4,4,TITAN V,2018,14,250,815,815,21100,18.177794,169.710334,190.344792,21.192104,208.23354,153185,17293,2999
5,5,TITAN RTX,2018,14,280,754,754,18600,16.078389,190.075574,208.426754,19.092700,252.08961,146590,20969,2499
6,6,RTX 3090Ti,2022,10,450,628,628,28300,15.412346,337.837085,357.301199,21.864204,537.95715,205459,22129,1999


In [8]:
desktop_gpu.head()

,Unnamed: 0,Product,Release Date,Process Size (nm),TDP (W),DieSizeValue,Die Size (mm^2),Transistors (million),Emb CFP,Ope CFP,Tot CFP,ECFPA,OCFPA,OpenCL Score,3D Mark Score,Price
0,0,TITAN BLACK,2014,28,230,561,561,7080,10.411002,100.750946,111.161948,8.359525,185.91248,30034,4179,999
1,1,TITAN X,2015,28,250,601,601,8000,11.833828,109.511898,121.345726,9.132249,188.62929,41486,5305,999
2,2,TITAN Pascal,2016,14,250,471,471,12000,9.513247,163.940411,173.453658,11.055738,360.31915,68312,9548,1199
3,3,TITAN Xp,2017,14,250,471,471,12000,9.513247,163.940411,173.453658,11.055738,360.31915,72482,10136,1199
4,4,TITAN V,2018,14,250,815,815,21100,27.487756,163.940411,191.428167,21.192104,208.23354,153185,17293,2999


In [10]:
processors = [server_cpu, server_gpu, desktop_cpu, desktop_gpu]

for processor in processors:
    processor['ECFPA'] = 1000 * processor['Emb CFP'].to_numpy(np.float32) / processor['DieSizeValue'].to_numpy(np.float32)
    processor['OCFPA'] = 1000 * processor['Ope CFP'].to_numpy(np.float32) / processor['DieSizeValue'].to_numpy(np.float32)


In [11]:
desktop_gpu.head()

,Unnamed: 0,Product,Release Date,Process Size (nm),TDP (W),DieSizeValue,Die Size (mm^2),Transistors (million),Emb CFP,Ope CFP,Tot CFP,ECFPA,OCFPA,OpenCL Score,3D Mark Score,Price
0,0,TITAN BLACK,2014,28,230,561,561,7080,10.411002,100.750946,111.161948,18.557936,179.591705,30034,4179,999
1,1,TITAN X,2015,28,250,601,601,8000,11.833828,109.511898,121.345726,19.690229,182.216141,41486,5305,999
2,2,TITAN Pascal,2016,14,250,471,471,12000,9.513247,163.940411,173.453658,20.197977,348.068848,68312,9548,1199
3,3,TITAN Xp,2017,14,250,471,471,12000,9.513247,163.940411,173.453658,20.197977,348.068848,72482,10136,1199
4,4,TITAN V,2018,14,250,815,815,21100,27.487756,163.940411,191.428167,33.727306,201.153885,153185,17293,2999
